Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [ ]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
import pandas as pd

df_train = pd.read_excel("/content/отзывы за лето (1).xls")

In [ ]:
df_train.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [ ]:
df_train['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Это ниразу не отзывы про лето!

### Предобработка

In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=8d4584bebe6b78daea8d1a34392c33247b84442244df81b1606d4a4e95f14885
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 29.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1da27c6f5d1b27dc53dbee8b3c2def69df5255fb6b3bee626aeddde4e1dde9ba
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train['Content'], df_train['Rating'], test_size=0.2, random_state=42)

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

X_train = X_train.apply(preprocess_text)

In [ ]:
X_train[:5]

2680                                  удобно
844        приходить смс код вход приложение
18714                         удобно понятно
13842    удобный приложение работать отлично
11974                     удобный приложение
Name: Content, dtype: object

In [ ]:
X_test = X_test.apply(preprocess_text)

In [ ]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]
tokens_filtered[:5] 

['удобно', 'приходить', 'смс', 'код', 'вход']

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [ ]:
x_train.shape

(16527, 40)

In [ ]:
max_len

40

In [ ]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 57, 45, 42, 18,  1], dtype=int32)

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
y_train[:5]

2680     5
844      2
18714    4
13842    5
11974    5
Name: Rating, dtype: int64

In [ ]:
num_classes = 6
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(64, 3, 1, activation='relu'))
model.add(Conv1D(64, 3, 2, activation='relu'))
model.add(Conv1D(128, 3, 1, activation='relu'))
model.add(Conv1D(128, 3, 2, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss', patience=5)  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/30
30/30 [==============================] - 5s 144ms/step - loss: 1.2605 - accuracy: 0.6818 - val_loss: 0.9674 - val_accuracy: 0.7120
Epoch 2/30
30/30 [==============================] - 4s 137ms/step - loss: 0.8979 - accuracy: 0.7268 - val_loss: 0.8649 - val_accuracy: 0.7344
Epoch 3/30
30/30 [==============================] - 4s 138ms/step - loss: 0.8473 - accuracy: 0.7410 - val_loss: 0.8377 - val_accuracy: 0.7411
Epoch 4/30
30/30 [==============================] - 4s 135ms/step - loss: 0.8333 - accuracy: 0.7464 - val_loss: 0.8309 - val_accuracy: 0.7417
Epoch 5/30
30/30 [==============================] - 4s 136ms/step - loss: 0.8271 - accuracy: 0.7490 - val_loss: 0.8445 - val_accuracy: 0.7411
Epoch 6/30
30/30 [==============================] - 4s 136ms/step - loss: 0.8240 - accuracy: 0.7504 - val_loss: 0.8300 - val_accuracy: 0.7423
Epoch 7/30
30/30 [==============================] - 4s 135ms/step - loss: 0.8193 - accuracy: 0.7516 - val_loss: 0.8297 - val_accuracy: 0.7441
Epoch 

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 0s 35ms/step - loss: 0.8826 - accuracy: 0.7406


Test score: 0.8825774192810059
Test accuracy: 0.7405614852905273


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

9/9 [==============================] - 0s 32ms/step


In [ ]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf

vectorizer = TextVectorization(max_tokens=200, output_sequence_length=40)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'приложение', 'удобно', 'работать']

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
#!wget https://rusvectores.org/ru/models/ruscorpora_upos_skipgram_300_10_2017 # я не нашел где там можно скачать эмбеддинги. Везде пишут что этот сайт для этого и создан, но не увидел ссылок на скачивание

--2022-12-13 13:04:45--  https://rusvectores.org/ru/models/ruscorpora_upos_skipgram_300_10_2017
Resolving rusvectores.org (rusvectores.org)... 172.104.228.108
Connecting to rusvectores.org (rusvectores.org)|172.104.228.108|:443... connected.
HTTP request sent, awaiting response... 308 PERMANENT REDIRECT
Location: https://rusvectores.org/ru/models/ruscorpora_upos_skipgram_300_10_2017/ [following]
--2022-12-13 13:04:45--  https://rusvectores.org/ru/models/ruscorpora_upos_skipgram_300_10_2017/
Reusing existing connection to rusvectores.org:443.
HTTP request sent, awaiting response... 303 SEE OTHER
Location: https://rusvectores.org/ru/ [following]
--2022-12-13 13:04:46--  https://rusvectores.org/ru/
Reusing existing connection to rusvectores.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 24948 (24K) [text/html]
Saving to: ‘ruscorpora_upos_skipgram_300_10_2017’

ruscorpora_upos_ski 100%[===================>]  24.36K  --.-KB/s    in 0.008s  

2022-12-13 13:04:46 (2.81 MB/s) 

In [ ]:
#path_to_glove_file = os.path.join(os.path.expanduser("~"), ".keras/datasets/glove.6B.100d.txt")

embeddings_index = {}
with open('/content/ruscorpora_upos_skipgram_300_10_2017') as f:
    for line in f:
      print(line)
      #word, coefs = line.split(maxsplit=1)
      #coefs = np.fromstring(coefs, "f", sep=" ")
      #embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

<!-- Look associates.html for more detailed comments -->

<!DOCTYPE html>

<html lang="ru">

<head>

  <title>RusVectōrēs: семантические модели для русского языка</title>

  <meta name="viewport" content="width=device-width, initial-scale=1.0">

  <meta charset="utf-8">

  <meta http-equiv="X-UA-Compatible" content="IE=edge">

  <meta name="description" content="РусВекторес: дистрибутивная семантика для русского языка, веб-интерфейс и модели для скачивания">

  <meta name='wmail-verification' content='070c0f36f9e585407648627b86eb5861' />

  <meta name="google-site-verification" content="4GNnqehZ8sQRJ2VCkAhJsFGB7fJJzc5DwiD9rkIQ0P4" />

  <meta property="og:type" content="website" />

  <meta property="og:site_name" content="RusVectores" />

  <meta property="og:url" content="https://rusvectores.org/ru/" />

  <meta property="og:title" content="RusVectōrēs: семантические модели для русского языка" />

  <meta property="og:description" content="РусВекторес: дистрибутивная семантика для ру

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)